In [1]:
import importlib
import data.VCTK
import model.model
import model.trainer
import torch

importlib.reload(data.VCTK) # prevent cache when making changes
importlib.reload(model.trainer) # prevent cache when making changes
importlib.reload(model.model) # prevent cache when making changes

<module 'model.model' from '/home/azureuser/dom/Project/model/model.py'>

In [2]:
dataset = data.VCTK.VCTKDataset("VCTK-Corpus-smaller/")

device = torch.device("cuda")
try:
    del myModel
except NameError:
    pass
myModel = model.model.SpectrogramModel().to(device)

In [33]:
import sys
import os.path
sys.path.append(os.path.abspath("ForwardTacotron"))
import models.fatchord_version
importlib.reload(models.fatchord_version)
try:
    del vocoderModel
except NameError:
    pass
vocoderModel = models.fatchord_version.WaveRNN(
    rnn_dims=512,
    fc_dims=512,
    bits=9, # OrigAuthor: bit depth of signal
    pad=2, # OrigAuthor: this will pad the input so that the resnet can 'see' wider than input length
    upsample_factors=(5, 5, 8), # OrigAuthor: NB - this needs to correctly factorise hop_length
    feat_dims=80,
    compute_dims=128,
    res_out_dims=128,
    res_blocks=10,
    hop_length=200,
    sample_rate=16000,
    mode="RAW", # OrigAuthor: either 'RAW' (softmax on raw bits) or 'MOL' (sample from mixture of logistics)
)

In [33]:
x = dataset[0][2]
x = x[0].unsqueeze(0)
print(x.shape)
x = x.repeat(50, 1, 1)
print(x.shape)

torch.Size([1, 80, 129])
torch.Size([50, 80, 129])


In [3]:
sum([len(x) for x in list(myModel.parameters())])

129184

In [53]:
torch.cuda.empty_cache()

In [5]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
def reasonable(n):
    return n/(1000**2)
print(reasonable(t),reasonable(r),reasonable(a),reasonable(f))

11996.954624 555.74528 295.996928 259.748352


In [22]:
del x

In [6]:
before = torch.cuda.memory_allocated(0)
x = dataset[0][2][0,:,:].to(device).repeat(100,1,1)
after = torch.cuda.memory_allocated(0)
print(f"Took {reasonable(after-before):1f}MB")
print(x.shape)

Took 4.128256MB
torch.Size([100, 80, 129])


/home/azureuser/dom/Project/data/VCTK.py:62: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168786/work/torch/csrc/utils/tensor_new.cpp:201.)
  return text, torch.tensor(clips), torch.tensor(spectros)


In [39]:
try:
    del myTrainer
except NameError:
    pass
importlib.reload(model.trainer) # prevent cache when making changes
myTrainer = model.trainer.Trainer(myModel, vocoderModel, device)

In [36]:
x.shape

torch.Size([100, 80, 129])

In [40]:
myTrainer.train_step(x, x)

RuntimeError: CUDA out of memory. Tried to allocate 1.66 GiB (GPU 0; 11.17 GiB total capacity; 9.57 GiB already allocated; 950.81 MiB free; 9.77 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
import tqdm.notebook as tqdm
for i, _ in tqdm.tqdm(list(enumerate(range(0,100)))):
    myTrainer.train_step(x, x)

  0%|          | 0/100 [00:00<?, ?it/s]

TypeError: conv1d() received an invalid combination of arguments - got (int, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!int!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!int!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)


In [17]:
import pathlib
import os.path
checkpoint_dir = pathlib.Path("checkpoints")
checkpoint_dir.mkdir(exist_ok=True)
model_path = checkpoint_dir / "model1.pth"
if os.path.exists(model_path):
    raise Exception("Won't overwrite existing models")
else:
    torch.save(myModel.state_dict(), model_path)

Exception: Won't overwrite existing models